<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=321f4196a19a6ac22483890f41c5e7ee861b29994b88c674149684e6029bcb48
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 12:19:59
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16


-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 26.86 K (0.18%)
Current PnL: -21.14 L (-13.85%)
CY Booked + Current PnL: -7.62 L (-4.99%)
-------------------
Total profit:  2.11 L
Total loss:  -23.25 L
-------------------
Total Booked + Current PnL: 19.36 L (15.41%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.29%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 87.81 L (60.35%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,102.77,53.0,M-SC,2.72,87397.0,-13380.0,13477.0,0.47,-13.28,15.42,0.09,245.0,-0.99,0.61,14.82,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,140.51,83.0,M-SC,3.21,157647.0,-16912.0,16947.0,3.15,-9.69,10.75,0.02,221.0,-1.00,1.09,37.13,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.36,88440.0,-2238.0,17149.0,-0.30,-2.47,19.39,16.44,101.0,-0.13,0.61,17.21,X40,ATH,PHARMA
34,ICICIGI,2252.93,-11.75,62.0,X-MC,7.57,190855.0,16535.0,18666.0,1.16,9.49,9.78,20.19,91.0,0.89,1.33,26.03,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.11,67.0,X-LC,8.76,301041.0,35615.0,21163.0,0.65,13.42,7.03,21.40,11.0,1.68,2.09,20.95,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.27,85.0,X-LC,7.97,252982.0,1214.0,51988.0,3.79,0.48,20.55,21.13,27.0,0.02,1.76,36.08,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,140.51,83.0,M-SC,3.21,157647.0,-16912.0,16947.0,3.15,-9.69,10.75,0.02,221.0,-1.00,1.09,37.13,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,27.55,75.0,X-SC,7.12,225787.0,34781.0,22917.0,2.76,18.21,10.15,30.21,157.0,1.52,1.57,42.89,X40N,NTT,FINANCE
13,BERGEPAINT,680.00,-13.70,69.0,X-MC,1.58,231883.0,4544.0,43524.0,2.22,2.00,18.77,21.14,106.0,0.10,1.61,30.70,XY24,NTT,PAINTS
64,SHALBY,327.00,1088.20,44.0,M-SC,16.47,161545.0,-19598.0,65070.0,1.89,-10.82,40.28,25.10,235.0,-0.30,1.12,29.46,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-55.29,52.0,H-SC,9.47,257706.0,-28799.0,286131.0,-5.13,-10.05,111.03,89.82,134.0,-0.10,1.79,36.35,XY24,NTT,FINANCE
15,CAMPUS,393.00,-26.15,40.0,M-SC,4.08,153744.0,-14031.0,69877.0,-1.96,-8.36,45.45,33.28,210.0,-0.20,1.07,19.16,XY24,NTT,FOOTWEAR
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.24,120494.0,-4803.0,63729.0,-1.59,-3.83,52.89,47.03,141.0,-0.08,0.84,50.87,MH,ATH,POWER
17,CERA,9475.00,-28.40,30.0,H-SC,2.27,130720.0,-45183.0,87203.0,-1.53,-25.69,66.71,23.89,149.0,-0.52,0.91,12.89,OX40N,NTT,CERAMICS
54,QUESS,986.00,-52.07,31.0,X-SC,37.26,47481.0,-17525.0,168453.0,-1.18,-26.96,354.78,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152365.0,1420.0,108042.0,0.22,0.94,70.91,72.51,53.0,0.01,1.06,7.02,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209221.0,-353.0,75027.0,-0.49,-0.17,35.86,35.63,NaN,-0.00,1.45,16.48,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,140.51,83.0,M-SC,3.21,157647.0,-16912.0,16947.0,3.15,-9.69,10.75,0.02,221.0,-1.00,1.09,37.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,102.77,53.0,M-SC,2.72,87397.0,-13380.0,13477.0,0.47,-13.28,15.42,0.09,245.0,-0.99,0.61,14.82,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,52.0,H-SC,2.40,224253.0,-45414.0,81740.0,1.48,-16.84,36.45,13.47,138.0,-0.56,1.56,14.03,XY24,NTT,PAINTS
66,SIS,528.0,2023.32,47.0,H-SC,2.14,85680.0,-25352.0,48958.0,-0.27,-22.83,57.14,21.26,156.0,-0.52,0.60,15.76,OX40N,NTT,MISC
72,TATAELXSI,9161.0,-24.89,47.0,H-MC,6.30,101716.0,-30119.0,72340.0,-0.64,-22.85,71.12,32.03,98.0,-0.42,0.71,13.93,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152365.0,1420.0,108042.0,0.22,0.94,70.91,72.51,53.0,0.01,1.06,7.02,XR,NTT,IT
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,57.0,M-SC,36.16,161793.0,12573.0,131602.0,1.58,8.43,81.34,96.62,208.0,0.10,1.12,75.09,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152365.0,1420.0,108042.0,0.22,0.94,70.91,72.51,53.0,0.01,1.06,7.02,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209221.0,-353.0,75027.0,-0.49,-0.17,35.86,35.63,NaN,-0.00,1.45,16.48,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.84,35.0,M-SC,9.16,112387.0,-2411.0,96451.0,-0.43,-2.10,85.82,81.92,223.0,-0.02,0.78,33.05,XR,NTT,DURABLES
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.24,120494.0,-4803.0,63729.0,-1.59,-3.83,52.89,47.03,141.0,-0.08,0.84,50.87,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-17.64,28.0,X-MC,5.78,252900.0,-996.0,83305.0,-0.93,-0.39,32.94,32.42,105.0,-0.01,1.76,14.65,X40,ATH,FMCG
1,ABB,7934.00,-43.47,30.0,H-LC,5.16,236400.0,-25219.0,144440.0,0.02,-9.64,61.10,45.57,7.0,-0.17,1.64,0.59,AR,NTT,ELECTRICAL
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87133.0,-41537.0,86837.0,-0.73,-32.28,99.66,35.20,219.0,-0.48,0.61,0.28,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.07,31.0,X-SC,37.26,47481.0,-17525.0,168453.0,-1.18,-26.96,354.78,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194205.0,-6615.0,74419.0,0.59,-3.29,38.32,33.76,80.0,-0.09,1.35,1.73,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.88,199047.0,-1091.0,21975.0,0.01,-0.55,11.04,10.44,4.0,-0.05,1.38,4.99,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194205.0,-6615.0,74419.0,0.59,-3.29,38.32,33.76,80.0,-0.09,1.35,1.73,X40,ATH,FMCG
65,SIEMENS,4671.50,-3.47,49.0,H-LC,1.62,155115.0,-30980.0,78457.0,1.50,-16.65,50.58,25.51,15.0,-0.39,1.08,14.31,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.60,45.0,X-MC,3.21,308301.0,-19690.0,126218.0,0.69,-6.00,40.94,32.48,92.0,-0.16,2.14,4.74,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-5.95,63.0,X-LC,3.28,246545.0,12401.0,36217.0,-0.15,5.30,14.69,20.77,86.0,0.34,1.71,12.58,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,31.0,X-SC,37.26,47481.0,-17525.0,168453.0,-1.18,-26.96,354.78,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87133.0,-41537.0,86837.0,-0.73,-32.28,99.66,35.20,219.0,-0.48,0.61,0.28,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.47,30.0,H-LC,5.16,236400.0,-25219.0,144440.0,0.02,-9.64,61.10,45.57,7.0,-0.17,1.64,0.59,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194205.0,-6615.0,74419.0,0.59,-3.29,38.32,33.76,80.0,-0.09,1.35,1.73,X40,ATH,FMCG
10,BAJAJHFL,181.50,-15.96,38.0,X-MC,6.42,176907.0,-23599.0,120757.0,-0.15,-11.77,68.26,48.45,90.0,-0.20,1.23,2.01,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,31.0,X-SC,37.26,47481.0,-17525.0,168453.0,-1.18,-26.96,354.78,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.75,42.0,X-SC,3.58,80812.0,-64348.0,140273.0,-1.09,-44.33,173.58,52.31,136.0,-0.46,0.56,8.17,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.36,88440.0,-2238.0,17149.0,-0.30,-2.47,19.39,16.44,101.0,-0.13,0.61,17.21,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87133.0,-41537.0,86837.0,-0.73,-32.28,99.66,35.20,219.0,-0.48,0.61,0.28,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.48,35.0,X-SC,0.97,106395.0,-21543.0,68678.0,-0.08,-16.84,64.55,36.84,143.0,-0.31,0.74,8.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,62.0,X-LC,8.77,293534.0,-52422.0,119116.0,-0.29,-15.15,40.58,19.28,1.0,-0.44,2.04,8.51,X40,ATH,IT
40,INFY,2275.00,-15.29,67.0,X-LC,4.32,330086.0,17198.0,154480.0,-0.13,5.50,46.80,54.87,3.0,0.11,2.29,14.23,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.88,199047.0,-1091.0,21975.0,0.01,-0.55,11.04,10.44,4.0,-0.05,1.38,4.99,X40,NTT,FMCG
81,VBL,671.64,-18.69,46.0,X-LC,5.85,292682.0,-23160.0,135834.0,-0.12,-7.33,46.41,35.67,5.0,-0.17,2.03,5.51,X40N,ATH,FMCG
1,ABB,7934.00,-43.47,30.0,H-LC,5.16,236400.0,-25219.0,144440.0,0.02,-9.64,61.10,45.57,7.0,-0.17,1.64,0.59,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,307.56,80.0,L-MC,6.68,50040.0,-27166.0,52557.0,1.56,-35.19,105.03,32.89,258.0,-0.52,0.35,37.81,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,47.0,L-SC,22.27,78528.0,-35021.0,75073.0,1.11,-30.84,95.60,35.27,268.0,-0.47,0.55,92.20,XR,NTT,HOTELS
7,ASIANTILES,137.00,7694.44,71.0,L-SC,15.94,85525.0,-8285.0,84901.0,0.15,-8.83,99.27,81.67,269.0,-0.10,0.59,65.62,XR,NTT,CERAMICS
50,MASFIN,398.61,-18.70,50.0,H-SC,7.52,92220.0,-5760.0,27362.0,-0.19,-5.88,29.67,22.05,152.0,-0.21,0.64,34.05,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.77,59.0,H-SC,4.77,132092.0,8756.0,122872.0,-0.49,7.10,93.02,106.72,119.0,0.07,0.92,32.14,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.27,85.0,X-LC,7.97,252982.0,1214.0,51988.0,3.79,0.48,20.55,21.13,27.0,0.02,1.76,36.08,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,140.51,83.0,M-SC,3.21,157647.0,-16912.0,16947.0,3.15,-9.69,10.75,0.02,221.0,-1.00,1.09,37.13,OX40N,NTT,PAINTS
79,VAIBHAVGBL,521.00,74.10,62.0,H-SC,7.59,150870.0,-31905.0,142979.0,0.98,-17.46,94.77,60.77,125.0,-0.22,1.05,35.87,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7694.44,71.0,L-SC,15.94,85525.0,-8285.0,84901.0,0.15,-8.83,99.27,81.67,269.0,-0.10,0.59,65.62,XR,NTT,CERAMICS
5,ANGELONE,3033.00,27.55,75.0,X-SC,7.12,225787.0,34781.0,22917.0,2.76,18.21,10.15,30.21,157.0,1.52,1.57,42.89,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.60
2,50,76.32


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.45
LC    28.01
MC    27.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.41
X40      20.85
X40N     12.40
XY25     10.18
XR        9.76
AR        9.11
OX40N     6.58
X200      1.84
SR        1.04
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.84
X-MC    22.98
X-LC    20.01
M-SC    12.52
X-SC     5.62
H-LC     4.56
H-MC     3.77
M-LC     2.45
L-SC     1.47
L-LC     0.99
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-3.98,37.61
IT,12.76,-14.35,75.56
FINANCE,12.06,-8.89,58.25
MISC,7.69,-17.52,76.80
PAINTS,6.02,-6.53,22.41
ELECTRICAL,5.60,-12.09,51.82
INSURANCE,4.52,2.64,31.67
PHARMA,3.47,0.12,33.39
AUTO,3.17,-52.48,115.61


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219007.0,20
AR,1260203.0,10
XR,1250888.0,13
X40,969257.0,14
X40N,682413.0,9
OX40N,538238.0,9
XY25,468406.0,7
SR,270381.0,2
MH,63729.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3399912.0,24
X-MC,1380144.0,16
M-SC,1240788.0,15
X-LC,769743.0,11
X-SC,540367.0,6
H-MC,329381.0,3
H-LC,281493.0,3
L-SC,249161.0,3
M-LC,153911.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1178201.0      6
           AR         839055.0      5
           XR         786340.0      7
M-SC       XY24       727482.0      6
X-MC       XY24       552522.0      4
           X40        483735.0      8
X-LC       X40        398685.0      5
X-SC       X40N       285077.0      4
H-SC       SR         270381.0      2
           OX40N      262206.0      3
X-LC       X40N       237434.0      3
H-LC       AR         222897.0      2
X-MC       XY25       183985.0      2
H-MC       XY24       182014.0      1
X-SC       XY24       168453.0      1
L-SC       XR         159974.0      2
X-MC       X40N       159902.0      2
M-SC       XR         143975.0      2
           XY25       131602.0      1
           AR         123224.0      2
           OX40N      114505.0      4
M-LC       XR         108042.0      1
L-SC       OX40N       89187.0      1
X-SC       X40         86837.0      1
H-MC       AR          75027.0      1
           OX40N       72340.0      1
X-LC       XY25        70752.0      2
H-SC       MH          63729.0      1
X-LC       XY24        62872.0      1
H-LC       X200        58596.0      1
L-MC       XR          52557.0      1
M-LC       XY25        45869.0      1
L-LC       XY25        36198.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
